In [4]:
# quantum_autoencoder_circuit.ipynb
# Jupyter notebook to create and draw a 4-qubit encoder-decoder quantum circuit using Qiskit

# Install Qiskit if not already installed
# !pip install qiskit matplotlib

import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.visualization import circuit_drawer

# --- Functions from your code ---

def create_ansatz(num_qubits, compression_level, param_prefix='θ'):
    total_qubits = num_qubits * 2 + 1
    qc = QuantumCircuit(total_qubits)
    num_parameters = 2 * sum(num_qubits - i for i in range(num_qubits - compression_level + 1))
    params = ParameterVector(param_prefix, num_parameters)
    param_index = 0
    for layer in range(num_qubits - compression_level + 1):
        for qubit in range(num_qubits - layer):
            qc.rx(params[param_index], qubit)
            param_index += 1
            qc.rz(params[param_index], qubit)
            param_index += 1
        if layer < num_qubits - compression_level:
            for qubit in range(num_qubits - layer - 1):
                qc.cx(qubit, qubit + 1)
    return qc, params


def create_reset_circuit(num_qubits, compression_level):
    total_qubits = num_qubits * 2 + 1
    reset_circuit = QuantumCircuit(total_qubits)
    for qubit in range(compression_level, num_qubits):
        reset_circuit.reset(qubit)
    return reset_circuit


def create_encoder_decoder_circuit(num_qubits, compression_level, decoder_option):
    if compression_level < 1 or compression_level > num_qubits:
        raise ValueError(f"Compression level must be between 1 and {num_qubits}")

    encoder, encoder_params = create_ansatz(num_qubits, compression_level, param_prefix='θ_enc')
    reset_circuit = create_reset_circuit(num_qubits, compression_level)

    if decoder_option == 1:
        decoder = encoder.inverse()
        complete_circuit = encoder.compose(reset_circuit).compose(decoder)
        return complete_circuit, encoder_params, None

    elif decoder_option == 2:
        decoder, decoder_params = create_ansatz(num_qubits, compression_level, param_prefix='θ_dec')
        decoder = decoder.reverse_ops()
        complete_circuit = encoder.compose(reset_circuit).compose(decoder)
        return complete_circuit, encoder_params, decoder_params

    else:
        raise ValueError("Invalid decoder option. Choose 1 for Qiskit's .inverse() or 2 for manual decoder.")

# --- Create and draw the circuit ---
num_qubits = 4
compression_level = 2
decoder_option = 1

circuit, enc_params, dec_params = create_encoder_decoder_circuit(num_qubits, compression_level, decoder_option)

# Draw the circuit using matplotlib
# Simple ASCII/text drawing
print(circuit.draw('text'))



     ┌──────────────┐┌──────────────┐     ┌──────────────┐ ┌──────────────┐»
q_0: ┤ Rx(θ_enc[0]) ├┤ Rz(θ_enc[1]) ├──■──┤ Rx(θ_enc[8]) ├─┤ Rz(θ_enc[9]) ├»
     ├──────────────┤├──────────────┤┌─┴─┐└──────────────┘┌┴──────────────┤»
q_1: ┤ Rx(θ_enc[2]) ├┤ Rz(θ_enc[3]) ├┤ X ├───────■────────┤ Rx(θ_enc[10]) ├»
     ├──────────────┤├──────────────┤└───┘     ┌─┴─┐      └───────────────┘»
q_2: ┤ Rx(θ_enc[4]) ├┤ Rz(θ_enc[5]) ├──────────┤ X ├──────────────■────────»
     ├──────────────┤├──────────────┤          └───┘            ┌─┴─┐      »
q_3: ┤ Rx(θ_enc[6]) ├┤ Rz(θ_enc[7]) ├───────────────────────────┤ X ├──────»
     └──────────────┘└──────────────┘                           └───┘      »
q_4: ──────────────────────────────────────────────────────────────────────»
                                                                           »
q_5: ──────────────────────────────────────────────────────────────────────»
                                                                           »

In [5]:
# quantum_autoencoder_fixed_ansatz.ipynb
# Jupyter notebook to create and draw a 4-qubit fixed ansatz encoder-decoder quantum circuit using Qiskit

# Install Qiskit if not already installed
# !pip install qiskit

from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit import ParameterVector

# --- Functions from your code ---

def create_ansatz(num_qubits=4, param_prefix='θ'):
    if num_qubits != 4:
        raise ValueError("This ansatz is fixed for 4 qubits.")

    total_params = 12  # 4 RX + 4 RZ + 4 CRX
    params = ParameterVector(param_prefix, total_params)
    
    q = QuantumRegister(num_qubits, 'q')
    qc = QuantumCircuit(q)

    # RX: params[0..3]
    for i in range(num_qubits):
        qc.rx(params[i], q[i])
    # RZ: params[4..7]
    for i in range(num_qubits):
        qc.rz(params[4 + i], q[i])
    # CRX: params[8..11]
    crx_pairs = [(3,0),(2,3),(1,2),(0,1)]
    for idx, (ctrl, targ) in enumerate(crx_pairs):
        qc.crx(params[8 + idx], q[ctrl], q[targ])

    return qc, params


def create_reset_circuit(num_qubits, compression_level):
    reset_circuit = QuantumCircuit(num_qubits)
    for qubit in range(compression_level, num_qubits):
        reset_circuit.reset(qubit)
    return reset_circuit


def create_encoder_decoder_circuit(num_qubits, compression_level, decoder_option):
    if compression_level < 1 or compression_level > num_qubits:
        raise ValueError(f"Compression level must be between 1 and {num_qubits}")

    encoder, encoder_params = create_ansatz(num_qubits, param_prefix='θ_enc')
    reset_circuit = create_reset_circuit(num_qubits, compression_level)

    if decoder_option == 1:
        decoder = encoder.inverse()
        complete_circuit = encoder.compose(reset_circuit).compose(decoder)
        return complete_circuit, encoder_params, None

    elif decoder_option == 2:
        decoder, decoder_params = create_ansatz(num_qubits, param_prefix='θ_dec')
        decoder = decoder.reverse_ops()
        complete_circuit = encoder.compose(reset_circuit).compose(decoder)
        return complete_circuit, encoder_params, decoder_params

    else:
        raise ValueError("Invalid decoder option. Choose 1 for inverse() or 2 for manual decoder.")


def update_circuit_parameters(circuit, encoder_params, decoder_params, new_angles):
    param_dict = {}
    encoder_flat = list(encoder_params)
    num_encoder_params = len(encoder_flat)
    param_dict.update(dict(zip(encoder_flat, new_angles[:num_encoder_params])))

    if decoder_params is not None:
        decoder_flat = list(decoder_params)
        param_dict.update(dict(zip(decoder_flat, new_angles[num_encoder_params:])))
    else:
        decoder_flat = []

    bound_circuit = circuit.assign_parameters(param_dict)
    return bound_circuit

# --- Create and draw the circuit ---
num_qubits = 4
compression_level = 2
decoder_option = 1

circuit, enc_params, dec_params = create_encoder_decoder_circuit(num_qubits, compression_level, decoder_option)

# Draw the circuit using text for compatibility
print(circuit.draw('text'))


     ┌──────────────┐┌──────────────┐┌──────────────┐                »
q_0: ┤ Rx(θ_enc[0]) ├┤ Rz(θ_enc[4]) ├┤ Rx(θ_enc[8]) ├────────────────»
     ├──────────────┤├──────────────┤└──────┬───────┘                »
q_1: ┤ Rx(θ_enc[1]) ├┤ Rz(θ_enc[5]) ├───────┼────────────────────────»
     ├──────────────┤├──────────────┤       │                        »
q_2: ┤ Rx(θ_enc[2]) ├┤ Rz(θ_enc[6]) ├───────┼───────────────■────────»
     ├──────────────┤├──────────────┤       │        ┌──────┴───────┐»
q_3: ┤ Rx(θ_enc[3]) ├┤ Rz(θ_enc[7]) ├───────■────────┤ Rx(θ_enc[9]) ├»
     └──────────────┘└──────────────┘                └──────────────┘»
«                                                                           »
«q_0: ─────────────────────────■────────────────■───────────────────────────»
«                      ┌───────┴───────┐┌───────┴────────┐                  »
«q_1: ────────■────────┤ Rx(θ_enc[11]) ├┤ Rx(-θ_enc[11]) ├────────■─────────»
«     ┌───────┴───────┐└───────────────┘└────────

In [6]:
# quantum_autoencoder_with_ancilla_draw.ipynb
# Jupyter notebook to create and draw the 4-qubit fixed ansatz encoder-decoder quantum circuit with ancilla qubits

# Install Qiskit if not already installed
# !pip install qiskit

from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit import ParameterVector

# --- Functions ---

def create_ansatz(num_qubits=4, param_prefix='θ'):
    if num_qubits != 4:
        raise ValueError("This ansatz is fixed for 4 qubits.")

    total_qubits = num_qubits * 2 + 1
    total_params = 12  # 4 RX + 4 RZ + 4 CRX
    params = ParameterVector(param_prefix, total_params)

    q = QuantumRegister(total_qubits, 'q')
    qc = QuantumCircuit(q)

    # RX and RZ on original qubits
    for i in range(num_qubits):
        qc.rx(params[i], q[i])
        qc.rz(params[4 + i], q[i])

    # CRX gates between original qubits
    crx_pairs = [(3,0),(2,3),(1,2),(0,1)]
    for idx, (ctrl, targ) in enumerate(crx_pairs):
        qc.crx(params[8 + idx], q[ctrl], q[targ])

    return qc, params


def create_reset_circuit(num_qubits, compression_level):
    total_qubits = num_qubits * 2 + 1
    reset_circuit = QuantumCircuit(total_qubits)
    for qubit in range(compression_level, num_qubits):
        reset_circuit.reset(qubit)
    return reset_circuit


def create_encoder_decoder_circuit(num_qubits, compression_level, decoder_option):
    encoder, encoder_params = create_ansatz(num_qubits, param_prefix='θ_enc')
    reset_circuit = create_reset_circuit(num_qubits, compression_level)

    if decoder_option == 1:
        decoder = encoder.inverse()
        complete_circuit = encoder.compose(reset_circuit).compose(decoder)
        return complete_circuit, encoder_params, None

    elif decoder_option == 2:
        decoder, decoder_params = create_ansatz(num_qubits, param_prefix='θ_dec')
        decoder = decoder.reverse_ops()
        complete_circuit = encoder.compose(reset_circuit).compose(decoder)
        return complete_circuit, encoder_params, decoder_params

    else:
        raise ValueError("Invalid decoder option. Choose 1 for inverse() or 2 for manual decoder.")

# --- Create and draw the circuit ---
num_qubits = 4
compression_level = 2
decoder_option = 1

circuit, enc_params, dec_params = create_encoder_decoder_circuit(num_qubits, compression_level, decoder_option)

# Draw the circuit using text output for Jupyter
print(circuit.draw('text'))

     ┌──────────────┐┌──────────────┐┌──────────────┐                »
q_0: ┤ Rx(θ_enc[0]) ├┤ Rz(θ_enc[4]) ├┤ Rx(θ_enc[8]) ├────────────────»
     ├──────────────┤├──────────────┤└──────┬───────┘                »
q_1: ┤ Rx(θ_enc[1]) ├┤ Rz(θ_enc[5]) ├───────┼────────────────────────»
     ├──────────────┤├──────────────┤       │                        »
q_2: ┤ Rx(θ_enc[2]) ├┤ Rz(θ_enc[6]) ├───────┼───────────────■────────»
     ├──────────────┤├──────────────┤       │        ┌──────┴───────┐»
q_3: ┤ Rx(θ_enc[3]) ├┤ Rz(θ_enc[7]) ├───────■────────┤ Rx(θ_enc[9]) ├»
     └──────────────┘└──────────────┘                └──────────────┘»
q_4: ────────────────────────────────────────────────────────────────»
                                                                     »
q_5: ────────────────────────────────────────────────────────────────»
                                                                     »
q_6: ────────────────────────────────────────────────────────────────»
      